In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
pd.set_option('future.no_silent_downcasting', True)

#### Getting the data

In [4]:
symbol = input("Enter the symbol you want to perform DCF on: ")

In [5]:
ticker = yf.Ticker(symbol)

In [6]:
selected_case = int(input("""
Choose 3 cases:
Case 1: Optimistic case
Case 2: Base case
Case 3: worst case
Enter your choice (1-3): """))

In [7]:
Balance_sheet = ticker.balance_sheet.transpose()
Income_statement = ticker.income_stmt.transpose()

In [8]:
Income_statement['Other revenue'] = Income_statement['Total Revenue'] - Income_statement['Cost Of Revenue']

In [9]:
Income_statement['EBT'] = Income_statement['EBIT'] - Income_statement['Interest Expense'] + Income_statement['Interest Income']

In [10]:
Income_statement['D&A'] = Income_statement['EBITDA'] - Income_statement['EBIT']

In [11]:
Income_statement['Taxes'] = Income_statement['EBT'] - Income_statement['Net Income']

#### P&L assumption

In [12]:
def forecaster(models: str, year_on_year: bool) -> pd.DataFrame:
    """
    Calculate the actual and forecast values for key financial metrics.

    Parameters:
    models (str): The financial models to include in the calculation, such as "Revenue", "COGS", "D&A", "Operating Expense", and "Interest Expense".
    year_on_year (bool): If True, the function will calculate the year-over-year change for each metric. If False, it will calculate the % change with respect to EBT/Revenue.

    Returns:
    pd.DataFrame: A dataframe containing the actual and forecast values for the specified financial metrics.

    Notes:
    - The "D&A" metric represents Depreciation and Amortization, which is the difference between EBITDA and EBIT.
    - The function assumes that the input data is from the Income Statement.
    """
    actual = pd.DataFrame(columns=sorted(list(Income_statement.index.year)))
    forecast = pd.DataFrame(columns=[x + max(list(Income_statement.index.year)) for x in range(1,6)]).transpose()

    # The use of model rather than models is as a naming convention - A shorter form
    if models == 'Revenues':
        revenues = pd.DataFrame(Income_statement['Operating Revenue']).rename({'Operating Revenue' : 'Revenues'},axis=1)
    elif models == 'COGS':
        model = 'Cost Of Revenue'
        revenues = pd.DataFrame(Income_statement['Cost Of Revenue'])
    else:
        revenues = pd.DataFrame(Income_statement[models])
    revenues.index = revenues.index.year
    revenues = revenues.transpose()

    actual_revenues = actual.copy()
    actual_revenues = pd.concat([actual_revenues, revenues])
    actual_revenues = actual_revenues.transpose()

    # forecasts items using year on year growth: Revenues, Other revenues
    if year_on_year:
        actual_revenues['y-o-y growth'] = actual_revenues[models].pct_change()

        if selected_case == 1:
            if models == 'Interest Expense': # Becasuse the best case for an expense is minimum
                forecast['y-o-y growth'] = min(actual_revenues['y-o-y growth'])
            else:
                forecast['y-o-y growth'] = max(actual_revenues['y-o-y growth'])
        elif selected_case == 2:
            forecast['y-o-y growth'] = (actual_revenues['y-o-y growth']).mean()
        elif selected_case == 3:
            forecast['y-o-y growth'] = (actual_revenues['y-o-y growth'].mean()) / 2

        forecasted = forecast.copy()
        forecasted[f'forecast {models}'] = actual_revenues[models].iloc[-1] * (1 + forecasted['y-o-y growth'])    
        
        for i in range(1, len(forecasted)):
            forecasted.loc[forecasted.index.min() + i,f'forecast {models}'] = forecasted.loc[forecasted.index.min() + i -1, f'forecast {models}'] * (1 + forecasted['y-o-y growth'].iloc[0])

        return forecasted   

    # forecasts items using % of revenue/ EBIT: COGS, Depreciation, Operating Expense
    else:
        revenue = pd.DataFrame(Income_statement['Operating Revenue']).sort_index()
        revenue.index = revenue.index.year


        try: 
            if models == 'Taxes':
                actual_revenues['% of EBT'] = actual_revenues[model] / Income_statement['EBT'].sort_index().values
            else:
                actual_revenues['% of revenue'] = actual_revenues[model] / revenue['Operating Revenue']    
        except Exception as e:
            if models == 'Taxes':
                actual_revenues['% of EBT'] = actual_revenues[models] / Income_statement['EBT'].sort_index().values
            else:
                actual_revenues['% of revenue'] = actual_revenues[models] / revenue['Operating Revenue']

        pct_type = 'EBT' if models == 'Taxes' else 'revenue'            

        if selected_case == 1:
            forecast[f'% of {pct_type}'] = min(actual_revenues[f'% of {pct_type}'])
        elif selected_case == 2:
            forecast[f'% of {pct_type}'] = (actual_revenues[f'% of {pct_type}']).mean()
        elif selected_case == 3:
            forecast[f'% of {pct_type}'] = (actual_revenues[f'% of {pct_type}'].mean()) / 2   

        if models == 'Taxes':
            forecasted = forecast.copy()  
            forecasted['Taxes'] = 0 # come back after balance sheet assumptions
        else:
            forecasted = forecast.copy()
            forecast_revenue = forecaster('Revenues', True)
            forecasted[f'forecast {models}'] = forecast_revenue['forecast Revenues'] * forecasted[f'% of {pct_type}']         
    
        return forecasted     

##### Modeling Revenue from sales and services

In [14]:
forecast_revenue = forecaster('Revenues', True)
forecast_revenue.to_csv('Data/Forecast_Revenue.csv')
forecast_revenue

,y-o-y growth,forecast Revenues
2024,0.141233,2.418443e+11
2025,0.141233,2.760007e+11
2026,0.141233,3.149810e+11
2027,0.141233,3.594667e+11
2028,0.141233,4.102351e+11


##### Modeling Other revenues


In [68]:
forecast_oth_revenue = forecaster('Other revenue',True)
forecast_oth_revenue

,y-o-y growth,forecast Other revenue
2024,0.07378,1.568277e+11
2025,0.07378,1.683985e+11
2026,0.07378,1.808229e+11
2027,0.07378,1.941640e+11
2028,0.07378,2.084894e+11


##### Modeling Cost of goods sold

In [15]:
forecast_Cogs = forecaster('COGS', False)
forecast_Cogs.to_csv('Data/forecast_COGS.csv')
forecast_Cogs

,% of revenue,forecast COGS
2024,0.314929,7.616370e+10
2025,0.314929,8.692051e+10
2026,0.314929,9.919653e+10
2027,0.314929,1.132063e+11
2028,0.314929,1.291948e+11


##### Modeling Depreciation and Amortizaiton

In [75]:
forecast_Depreciation = forecaster('D&A', False)
forecast_Depreciation

,% of revenue,forecast D&A
2024,0.037167,8.432419e+09
2025,0.037167,9.027886e+09
2026,0.037167,9.665403e+09
2027,0.037167,1.034794e+10
2028,0.037167,1.107867e+10


##### Modeling Operating Expense

In [71]:
forecast_Expense = forecaster('Operating Expense', False)
forecast_Expense

,% of revenue,forecast Operating Expense
2024,0.139469,3.164266e+10
2025,0.139469,3.387716e+10
2026,0.139469,3.626944e+10
2027,0.139469,3.883065e+10
2028,0.139469,4.157273e+10


##### Modeling Interest Expense

In [72]:
forecast_Interest = forecaster('Interest Expense', True)
forecast_Interest

,y-o-y growth,forecast Interest Expense
2024,-0.04354,1.882314e+09
2025,-0.04354,1.800358e+09
2026,-0.04354,1.721971e+09
2027,-0.04354,1.646997e+09
2028,-0.04354,1.575287e+09


##### Modeling Taxes

In [73]:
forecast_taxes = forecaster('Taxes', False)
forecast_taxes

,% of EBT,Taxes
2024,0.092123,0
2025,0.092123,0
2026,0.092123,0
2027,0.092123,0
2028,0.092123,0
